In [ ]:
import datetime, dateutil, json, os, pickle, requests, time
from bs4 import BeautifulSoup
import pandas as pd
from twilio.rest import Client

In [ ]:
def get_form_values(form):
    return {i.get("name"):(i.get("value") or "") for i in form.find_all(["input", "select"])}

def parse_attendance(bs: BeautifulSoup):
    tables = bs.find_all("table")
    bs_table = tables[0]

    row_names = []
    col_names = []
    rows = []

    for r, bs_row in enumerate(bs_table.find_all(["th", "tr"])):
        if r > 0:
            rows.append([])
        for c, elt in enumerate(bs_row.find_all("td")):
            if r == 0 and c == 0:
                pass
            elif r == 0:
                col_names.append(dateutil.parser.parse(elt.text.split()[1]))
            elif c == 0:
                lines = elt.text.splitlines()
                row_names.append(f"P{lines[5].split()[1]} {lines[2].strip()} {lines[8].strip()}")
            else:
                spans = elt.select("div div span:not(.sg-assignment-description):not(.sg-right)")
                if spans:
                    e = spans[0].text.strip()
                else:
                    e = ""
                rows[-1].append(e)

    return pd.DataFrame(rows, columns=col_names, index=row_names)
    
def send_sms(body):
    # Download the helper library from https://www.twilio.com/docs/python/install

    twilio_config = json.load(open("secrets/twilio.json"))
    client = Client(twilio_config["AccountSID"], twilio_config["AuthToken"])

    for dest_phone_number in twilio_config["DestPhoneNumbers"]:
        message = client.messages.create(
            body=body,
            from_=twilio_config["TwilioPhoneNumber"],
            to=dest_phone_number)
        time.sleep(1.1)

def readable_date(timestamp):
    date = timestamp.date()
    today = datetime.date.today()
    days_old = (today - date).days
    if days_old == 0:
        return f"Today ({date.strftime('%a')})"
    elif days_old == 1:
        return f"Yesterday ({date.strftime('%a')})"
    else:
        return f"{date.strftime('%a %m/%d')}"

In [ ]:
def log_in_and_get_home_page():
    print("Logging in")
    login_url = "https://hac40.pps.k12.pa.us/HomeAccess4_1/Account/LogOn?ReturnUrl=%2fHomeAccess4_1%2fHome%2fWeekView"
    html = session.get(login_url).text
    bs = BeautifulSoup(html, "html.parser")
    forms = bs.find_all("form")
    form = forms[0]

    form_vals = get_form_values(form)

    login = json.load(open("secrets/login.json"))

    form_vals["LogOnDetails.UserName"] = login["username"]
    form_vals["LogOnDetails.Password"] = login["password"]
    form_vals["Database"] = "10"

    login_post_url =  "https://hac40.pps.k12.pa.us/HomeAccess4_1/Account/LogOn?ReturnUrl=%2fHomeAccess4_1%2fHome%2fWeekView"
    return session.post(login_post_url, data=form_vals, headers={"Referer": login_url})

session = requests.Session()


In [ ]:
resp = log_in_and_get_home_page()
att_df = parse_attendance(BeautifulSoup(resp.text, "html.parser"))
att_df

In [ ]:
os.makedirs("data", exist_ok=True)
week_file = att_df.columns[0].date().strftime("data/%Y-%m-%d.pickle")
if os.path.exists(week_file):
    prev_df = pd.read_pickle(week_file)
else:
    prev_df = att_df.copy()
    prev_df.loc[:,:] = ""
diffs = []
for (idxRow, s1), (_, s2) in zip(prev_df.iterrows(), att_df.iterrows()):
    for (idxCol, v1), (_, v2) in zip(s1.items(), s2.items()):
        if v1 != v2:
            if v1 == "":
                diffs.append(f"{v2} for {readable_date(idxCol)} {idxRow}")
            elif v2 == "":
                diffs.append(f"Removed: {v1} removed from {readable_date(idxCol)} {idxRow}")
            else:
                diffs.append(f"Changed: {v2} (was {v1}) for {readable_date(idxCol)} {idxRow}")
if diffs:
    message = "\n".join(reversed(diffs))
    print(f"Sending sms: {diffs}")
    send_sms(message)
else:
    print("No attendance changes")

temp_name = f"{week_file}.tmp{os.getpid()}"
att_df.to_pickle(temp_name)
os.rename(temp_name, week_file)